In [66]:
#Importing libraries needed
import requests
import pandas as pd

In [67]:
#Request in ibge API
r = requests.get('https://servicodados.ibge.gov.br/api/v1/localidades/municipios')

In [68]:
#Code status of request
r.status_code

200

In [69]:
#Reading json and converting to dataframe
data = r.json()
df = pd.json_normalize(data)

In [72]:
df

,id,nome,microrregiao.id,microrregiao.nome,microrregiao.mesorregiao.id,microrregiao.mesorregiao.nome,microrregiao.mesorregiao.UF.id,microrregiao.mesorregiao.UF.sigla,microrregiao.mesorregiao.UF.nome,microrregiao.mesorregiao.UF.regiao.id,...,regiao-imediata.id,regiao-imediata.nome,regiao-imediata.regiao-intermediaria.id,regiao-imediata.regiao-intermediaria.nome,regiao-imediata.regiao-intermediaria.UF.id,regiao-imediata.regiao-intermediaria.UF.sigla,regiao-imediata.regiao-intermediaria.UF.nome,regiao-imediata.regiao-intermediaria.UF.regiao.id,regiao-imediata.regiao-intermediaria.UF.regiao.sigla,regiao-imediata.regiao-intermediaria.UF.regiao.nome
0,1100015,Alta Floresta D'Oeste,11006,Cacoal,1102,Leste Rondoniense,11,RO,Rondônia,1,...,110005,Cacoal,1102,Ji-Paraná,11,RO,Rondônia,1,N,Norte
1,1100023,Ariquemes,11003,Ariquemes,1102,Leste Rondoniense,11,RO,Rondônia,1,...,110002,Ariquemes,1101,Porto Velho,11,RO,Rondônia,1,N,Norte
2,1100031,Cabixi,11008,Colorado do Oeste,1102,Leste Rondoniense,11,RO,Rondônia,1,...,110006,Vilhena,1102,Ji-Paraná,11,RO,Rondônia,1,N,Norte
3,1100049,Cacoal,11006,Cacoal,1102,Leste Rondoniense,11,RO,Rondônia,1,...,110005,Cacoal,1102,Ji-Paraná,11,RO,Rondônia,1,N,Norte
4,1100056,Cerejeiras,11008,Colorado do Oeste,1102,Leste Rondoniense,11,RO,Rondônia,1,...,110006,Vilhena,1102,Ji-Paraná,11,RO,Rondônia,1,N,Norte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,5222005,Vianópolis,52016,Pires do Rio,5205,Sul Goiano,52,GO,Goiás,5,...,520002,Anápolis,5201,Goiânia,52,GO,Goiás,5,CO,Centro-Oeste
5566,5222054,Vicentinópolis,52015,Meia Ponte,5205,Sul Goiano,52,GO,Goiás,5,...,520009,Piracanjuba,5202,Itumbiara,52,GO,Goiás,5,CO,Centro-Oeste
5567,5222203,Vila Boa,52012,Entorno de Brasília,5204,Leste Goiano,52,GO,Goiás,5,...,520022,Flores de Goiás,5206,Luziânia - Águas Lindas de Goiás,52,GO,Goiás,5,CO,Centro-Oeste
5568,5222302,Vila Propício,52012,Entorno de Brasília,5204,Leste Goiano,52,GO,Goiás,5,...,520018,Ceres - Rialma - Goianésia,5205,Porangatu - Uruaçu,52,GO,Goiás,5,CO,Centro-Oeste


In [73]:
#Choosing columns I am interested
df_selected = df[['id', 'nome', 'microrregiao.mesorregiao.nome', 'microrregiao.mesorregiao.UF.sigla', 'microrregiao.mesorregiao.UF.nome', 'regiao-imediata.regiao-intermediaria.UF.regiao.nome']]

In [74]:
#Renaming columns
df_selected.rename(columns={'nome':'city', 
        'microrregiao.mesorregiao.nome':'mesoregion', 
        'microrregiao.mesorregiao.UF.sigla':'state_abbreviation', 
        'microrregiao.mesorregiao.UF.nome':'state',
        'regiao-imediata.regiao-intermediaria.UF.regiao.nome':'region'}, inplace=True)

/tmp/ipykernel_656/856496671.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected.rename(columns={'nome':'city',


In [75]:
df_selected

,id,city,mesoregion,state_abbreviation,state,region
0,1100015,Alta Floresta D'Oeste,Leste Rondoniense,RO,Rondônia,Norte
1,1100023,Ariquemes,Leste Rondoniense,RO,Rondônia,Norte
2,1100031,Cabixi,Leste Rondoniense,RO,Rondônia,Norte
3,1100049,Cacoal,Leste Rondoniense,RO,Rondônia,Norte
4,1100056,Cerejeiras,Leste Rondoniense,RO,Rondônia,Norte
...,...,...,...,...,...,...
5565,5222005,Vianópolis,Sul Goiano,GO,Goiás,Centro-Oeste
5566,5222054,Vicentinópolis,Sul Goiano,GO,Goiás,Centro-Oeste
5567,5222203,Vila Boa,Leste Goiano,GO,Goiás,Centro-Oeste
5568,5222302,Vila Propício,Leste Goiano,GO,Goiás,Centro-Oeste


In [77]:
#Creating a new column to make some treatments in city names to became in upper case and without special characters.
df_selected['city_padr'] = df_selected['city'].str.upper()

/tmp/ipykernel_656/1210719470.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['city_padr'] = df_selected['city'].str.upper()


In [78]:
letter_changes = {
                    'Ã': 'A', 
                    'Á': 'A',
                    'À': 'A',
                    'Â': 'A',
                    'Ä': 'A',
                    'É': 'E',
                    'È': 'E',
                    'Ê': 'E',
                    'Ë': 'E',
                    'Í': 'I',
                    'Ì': 'I',
                    'Î': 'I',
                    'Ï': 'I',
                    'Õ': 'O',
                    'Ó': 'O',
                    'Ò': 'O',
                    'Ô': 'O',
                    'Ö': 'O',
                    'Ú': 'U',
                    'Ù': 'U',
                    'Û': 'U',
                    'Ü': 'U',
                    'Ç': 'C',
                    "'": ' ',
                    '-': ' '}
df_selected['city_padr'] = df_selected.city_padr.replace(letter_changes, regex=True)

/tmp/ipykernel_656/4104093980.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['city_padr'] = df_selected.city_padr.replace(letter_changes, regex=True)


In [79]:
df_selected

,id,city,mesoregion,state_abbreviation,state,region,city_padr
0,1100015,Alta Floresta D'Oeste,Leste Rondoniense,RO,Rondônia,Norte,ALTA FLORESTA D OESTE
1,1100023,Ariquemes,Leste Rondoniense,RO,Rondônia,Norte,ARIQUEMES
2,1100031,Cabixi,Leste Rondoniense,RO,Rondônia,Norte,CABIXI
3,1100049,Cacoal,Leste Rondoniense,RO,Rondônia,Norte,CACOAL
4,1100056,Cerejeiras,Leste Rondoniense,RO,Rondônia,Norte,CEREJEIRAS
...,...,...,...,...,...,...,...
5565,5222005,Vianópolis,Sul Goiano,GO,Goiás,Centro-Oeste,VIANOPOLIS
5566,5222054,Vicentinópolis,Sul Goiano,GO,Goiás,Centro-Oeste,VICENTINOPOLIS
5567,5222203,Vila Boa,Leste Goiano,GO,Goiás,Centro-Oeste,VILA BOA
5568,5222302,Vila Propício,Leste Goiano,GO,Goiás,Centro-Oeste,VILA PROPICIO


In [80]:
#Finals table download
df_selected.to_csv('regioes_ibge.csv', index=False)
df_selected.to_excel("regioes_ibge.xlsx", index=False)